# Data Integrity Checks

# Make sure experiment run events are as expected

In [1]:
import sqlite3
import pandas as pd
from util import getTaskFromLocation
import sqlite3
import pandas as pd
from util import getTaskFromLocation, getTaskDurationSeconds
from datetime import datetime

expectedEvents = [
    ("TASKSTART", 1),
    ("CODEREVEALED", 1),
    ("TASKCOMPLETED", 1),
    ("TASKSTART", 2),
    ("CODEREVEALED", 2),
    ("TASKCOMPLETED", 2),
]

with sqlite3.connect('data/data_anon.db') as connection:
    # Get relevant events, aka those that we use to compute other values in utils
    df = pd.read_sql_query("SELECT * FROM events WHERE type in ('TASKSTART', 'CODEREVEALED', 'TASKCOMPLETED')", connection)
    df['task'] = df['location'].map(getTaskFromLocation)

for experimentid in df['experiment'].unique().tolist():
    print(f'❔ Checking {experimentid}')
    dfExperiment = df[(df['experiment'] == experimentid) & (df['task'] != 0)].reset_index()

    if (len(expectedEvents) != dfExperiment.shape[0]):
        print(f'🔴 Expected {len(expectedEvents)} events, experiment {experimentid} has {dfExperiment.shape[0]} events.')
        print(dfExperiment[['type', 'task']])
        continue

    print(f'✅ Experiment {experimentid} has the expected amount of events.')

    i = 0
    for expectedEvent, task in expectedEvents:
        if (dfExperiment.loc[i, 'type'] != expectedEvent or task != dfExperiment.loc[i, 'task']):
            print(f'🔴 Expected {expectedEvent} and task {task}, experiment {experimentid} has {dfExperiment.loc[i, "type"]} and task {dfExperiment.loc[i, "task"]}.')
            print(dfExperiment[['type', 'task']])
            break
        i += 1
    
    print(f'\t✅ All events for experiment {experimentid} are in the expected order.')
print(f'✅ Checked all experiments for expected amount of events and order of events.')


❔ Checking 159
✅ Experiment 159 has the expected amount of events.
	✅ All events for experiment 159 are in the expected order.
❔ Checking 124
✅ Experiment 124 has the expected amount of events.
	✅ All events for experiment 124 are in the expected order.
❔ Checking 133
✅ Experiment 133 has the expected amount of events.
	✅ All events for experiment 133 are in the expected order.
❔ Checking 83
✅ Experiment 83 has the expected amount of events.
	✅ All events for experiment 83 are in the expected order.
❔ Checking 163
✅ Experiment 163 has the expected amount of events.
	✅ All events for experiment 163 are in the expected order.
❔ Checking 13
✅ Experiment 13 has the expected amount of events.
	✅ All events for experiment 13 are in the expected order.
❔ Checking 55
✅ Experiment 55 has the expected amount of events.
	✅ All events for experiment 55 are in the expected order.
❔ Checking 82
✅ Experiment 82 has the expected amount of events.
	✅ All events for experiment 82 are in the expected ord

# Make sure time tracking worked as expected

In [2]:
def getTaskDurationFromCreated(experimentId: str, taskId: int, df: pd.DataFrame):
    start = df[
        (df["experiment"] == experimentId)
        & (df["task"] == taskId)
        & (df["type"] == "CODEREVEALED")
    ]
    finish = df[
        (df["experiment"] == experimentId)
        & (df["task"] == taskId)
        & (df["type"] == "TASKCOMPLETED")
    ]

    if len(start.index) == 0 or len(finish.index) == 0:
        return None
    
    return (
        datetime.strptime(finish.iloc[0]["created"], '%Y-%m-%d %H:%M:%S.%fZ').timestamp()
        - datetime.strptime(start.iloc[0]["created"], '%Y-%m-%d %H:%M:%S.%fZ').timestamp()
    )

with sqlite3.connect('data/data_anon.db') as connection:
    # Get relevant events, aka those that we use to compute other values in utils
    df = pd.read_sql_query("SELECT * FROM events WHERE type in ('TASKSTART', 'CODEREVEALED', 'TASKCOMPLETED')", connection)
    df['task'] = df['location'].map(getTaskFromLocation)

df = df[['created', 'experiment', 'task', 'time', 'type']]

print("Printing all time differences between times from the client and when events reached the server larger than 10ms.")

for experiment in df['experiment'].unique().tolist():
    for task in df['task'].unique().tolist():
        taskDurationSecondsFromTime = getTaskDurationSeconds(experimentId=experiment, taskId=task, df=df)
        taskDurationSecondsFromCreated = getTaskDurationFromCreated(experimentId=experiment, taskId=task, df=df)

        difference = abs(taskDurationSecondsFromTime - taskDurationSecondsFromCreated)

        if (difference > 0.01):
            print(f'{difference * 1000}ms difference in {experiment}, task {task}.')

print("✅ Assuming network lag did not majorly influence time measurement.")

Printing all time differences between times from the client and when events reached the server larger than 10ms.
10.999801635762196ms difference in 159, task 2.
10.000074386596225ms difference in 133, task 1.
18.999910354636995ms difference in 43, task 1.
15.000032424921983ms difference in 102, task 2.
10.000110626208425ms difference in 1, task 2.
44.99990463256154ms difference in 99, task 1.
10.999950408944414ms difference in 78, task 2.
11.0001106262132ms difference in 76, task 2.
18.999816894506694ms difference in 11, task 2.
15.999868392952976ms difference in 7, task 1.
10.99997138977038ms difference in 84, task 0.
14.999967575079154ms difference in 84, task 1.
12.000127792362036ms difference in 128, task 1.
✅ Assuming network lag did not majorly influence time measurement.


# Make sure all values are expected

In [3]:
df = pd.read_csv('./data/generated/results_anon.csv', dtype = {'student_id_anon':str})

print(f'❔ Verifying no duplicate values')
if len(df['student_id_anon']) != df['student_id_anon'].nunique():
    print(f'🔴 Expected no duplicate values, found {df.isnull().sum().sum()} duplicate values.')
else:
    print(f'✅ Found no duplicate values.')

# We don't verify strings
df = df.drop(columns=['seq', 'student_id_anon'])

print(f'❔ Verifying no NaN values')
if df.isnull().sum().sum() > 0:
    print(f'🔴 Expected no NaN values, found {df.isnull().sum().sum()} NaN values.')
else:
    print(f'✅ Found no NaN values.')

print(f'❔ Verifying no negative values')
if (df < 0).any().any() > 0:
    print(f'🔴 Expected no negative values, found {(df < 0).any().any()} negative values.')
else:
    print(f'✅ Found no negative values.')

print(f'❔ Verifying no values > 1 for correctness')
if (df[['t1_correctness', 't2_correctness']] > 1).any().any() > 0:
    print(f'🔴 Expected no values > 1 for correctness, found values > 1 for correctness.')
else:
    print(f'✅ Found no values > 1 for correctness.')

print(f'❔ Verifying no values > 20min for combined time')
overTime = (df['t1_time'] + df['t2_time'] > 20 * 60).any().any()
if (overTime > 0):
    print(f'🔴 Expected no values > 20min for combined time, found {overTime.sum()} values > 20min for combined time.')
    print(f'🔴 Maximum time: {(df["t1_time"] + df["t2_time"]).max() / 60} minutes.')
    print(f'✅ Considered okay and likely related to delay after being told about running out of time.')
else:
    print(f'✅ Found no values > 20min for combined time.')
    

❔ Verifying no duplicate values
✅ Found no duplicate values.
❔ Verifying no NaN values
✅ Found no NaN values.
❔ Verifying no negative values
✅ Found no negative values.
❔ Verifying no values > 1 for correctness
✅ Found no values > 1 for correctness.
❔ Verifying no values > 20min for combined time
🔴 Expected no values > 20min for combined time, found 1 values > 20min for combined time.
🔴 Maximum time: 20.022033333333333 minutes.
✅ Considered okay and likely related to delay after being told about running out of time.


# Make sure no outliers exist in times or correctness

In [4]:
outlierDetectionColumns = [
    't1_time',
    't1_correctness',
    't2_time',
    't2_correctness',
]

df = pd.read_csv('./data/generated/results_anon.csv', dtype = {'student_id_anon':str})

df['has_outlier'] = False

for column in outlierDetectionColumns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)

    # Calculate IQR
    IQR = Q3 - Q1

    # Determine the lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df[f'{column}_outlier'] = (df[column] < lower_bound) | (df[column] > upper_bound)

outlierColumns = [f"{col}_outlier" for col in outlierDetectionColumns]

df['has_outlier'] = df[outlierColumns].any(axis=1)

df[df['has_outlier'] == False].to_csv('./data/generated/results_anon_without_outliers.csv', index=False)
df[df['has_outlier'] == True].to_csv('./data/generated/results_anon_only_outliers.csv', index=False)

print(f'🔴 {df[df["has_outlier"] == True].shape[0]} experiments have values that are outliers according to 1.5 IQR, saved to /data/generated/results_anon_only_outliers.csv.')
print(f'✅ Removing these outliers does not change the results of the hypotheses tests, the only change is that t2_time is normal distributed without outliers.')

🔴 6 experiments have values that are outliers according to 1.5 IQR, saved to /data/generated/results_anon_only_outliers.csv.
✅ Removing these outliers does not change the results of the hypotheses tests, the only change is that t2_time is normal distributed without outliers.
